In [37]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
import pandas as pd
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import StandardScaler, LabelBinarizer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_predict
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers
from keras import optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop
from tensorflow.keras.metrics import binary_crossentropy, categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
import cv2 as cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight 
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
import time
import scipy
from scipy import ndimage
from sklearn.metrics import confusion_matrix, classification_report
from IPython.core.display import display, HTML
# stop annoying tensorflow warning messages
import logging
import splitfolders
logging.getLogger("tensorflow").setLevel(logging.ERROR)
np.random.seed(123)
print ('modules loaded')


modules loaded


In [2]:
data_training_ALL_dir = 'data/training_data/ALL/'
data_training_Normal_dir = 'data/training_data/Normal/'
new_dir = 'data/split/'

In [3]:
imgs_ALL = [file for file in os.listdir(data_training_ALL_dir) if file.endswith('.bmp')]
imgs_ALL[0:10]

['UID_51_26_4_all.bmp',
 'UID_20_1_1_all.bmp',
 'UID_46_16_4_all.bmp',
 'UID_23_7_2_all.bmp',
 'UID_4_4_2_all.bmp',
 'UID_45_30_2_all.bmp',
 'UID_28_2_3_all.bmp',
 'UID_35_8_7_all.bmp',
 'UID_45_25_8_all.bmp',
 'UID_51_14_6_all.bmp']

In [4]:
print('There are', len(imgs_ALL), 'ALL images')

There are 7272 ALL images


In [5]:
imgs_Normal = [file for file in os.listdir(data_training_Normal_dir) if file.endswith('.bmp')]
print('There are', len(imgs_Normal), 'Normal images')

There are 3389 Normal images


In [6]:
# os.mkdir(new_dir)

train_folder = os.path.join(new_dir, 'train')
train_ALL = os.path.join(train_folder, 'ALL')
train_Normal = os.path.join(train_folder, 'Normal')

val_folder = os.path.join(new_dir, 'validation')
val_ALL = os.path.join(val_folder, 'ALL')
val_Normal = os.path.join(val_folder, 'Normal')

In [7]:
train_ALL

'data/split/train/ALL'

In [8]:
# os.mkdir(train_folder)
# os.mkdir(train_ALL)
# os.mkdir(train_Normal)

# os.mkdir(val_folder)
# os.mkdir(val_ALL)
# os.mkdir(val_Normal)

In [9]:
# train ALL
imgs = imgs_ALL[:5700]
for img in imgs:
    origin = os.path.join(data_training_ALL_dir, img)
    destination = os.path.join(train_ALL, img)
    shutil.copyfile(origin, destination)
    
# validation ALL
imgs = imgs_ALL[5700:]
for img in imgs:
    origin = os.path.join(data_training_ALL_dir, img)
    destination = os.path.join(val_ALL, img)
    shutil.copyfile(origin, destination)
    
# train Normal
imgs = imgs_Normal[:2700]
for img in imgs:
    origin = os.path.join(data_training_Normal_dir, img)
    destination = os.path.join(train_Normal, img)
    shutil.copyfile(origin, destination)

# validation Normal
imgs = imgs_Normal[2700:]
for img in imgs:
    origin = os.path.join(data_training_Normal_dir, img)
    destination = os.path.join(val_Normal, img)
    shutil.copyfile(origin, destination)

In [10]:
print('There are', len(os.listdir(train_ALL)), 'ALL images in the training set')
print('There are', len(os.listdir(train_Normal)), 'Normal images in the training set')
print('There are', len(os.listdir(val_ALL)), 'ALL images in the validation set')
print('There are', len(os.listdir(val_Normal)), 'Normal images in the validation set')

There are 5700 ALL images in the training set
There are 2700 Normal images in the training set
There are 1572 ALL images in the validation set
There are 689 Normal images in the validation set


In [11]:
# get all the data in the directory split/train (542 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_folder,
        class_mode='binary',
        target_size=(225, 225), batch_size=8400)

# get all the data in the directory split/validation (200 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_folder,
        class_mode='binary',
        target_size=(225, 225), batch_size = 2261)

Found 8400 images belonging to 2 classes.
Found 2261 images belonging to 2 classes.


In [12]:
train_generator.class_indices

{'ALL': 0, 'Normal': 1}

In [13]:
val_generator.class_indices

{'ALL': 0, 'Normal': 1}

In [14]:
# create the data sets
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)

In [15]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
# m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
# print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
# print ("test_images shape: " + str(test_images.shape))
# print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 8400
Number of validation samples: 2261
train_images shape: (8400, 225, 225, 3)
train_labels shape: (8400,)
val_images shape: (2261, 225, 225, 3)
val_labels shape: (2261,)


In [16]:
train_img = train_images.reshape(train_images.shape[0], -1)
# test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
# print(test_img.shape)
print(val_img.shape)

(8400, 151875)
(2261, 151875)


In [17]:
# train_y = np.reshape(train_labels[:,0], (8400,1))
# # test_y = np.reshape(test_labels[:,0], (180,1))
# val_y = np.reshape(val_labels[:,0], (2261,1))

In [18]:
# train_y

In [19]:
test_df = pd.read_csv('data/test_data/test_data_labels.csv')

In [20]:
test_df.head()

,Patient_ID,new_names,labels
0,UID_57_29_1_all.bmp,1.bmp,1
1,UID_57_22_2_all.bmp,2.bmp,1
2,UID_57_31_3_all.bmp,3.bmp,1
3,UID_H49_35_1_hem.bmp,4.bmp,0
4,UID_58_6_13_all.bmp,5.bmp,1


In [21]:
test_df = test_df.replace({'labels':{0 : 'ALL', 1 : 'Normal'}})

# binarizer = LabelBinarizer()
# test_df.labels = binarizer.fit_transform(test_df.labels)

test_df = test_df.drop('Patient_ID', axis=1)

test_df.head(20)

,new_names,labels
0,1.bmp,Normal
1,2.bmp,Normal
2,3.bmp,Normal
3,4.bmp,ALL
4,5.bmp,Normal
5,6.bmp,Normal
6,7.bmp,ALL
7,8.bmp,ALL
8,9.bmp,Normal
9,10.bmp,Normal


In [22]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_dataframe(test_df,
                                         directory='data/test_data',
                                         x_col='new_names',
                                         y_col='labels',
                                         class_mode='binary',
                                         target_size=(225, 225),
                                         batch_size=1867,)

Found 1867 validated image filenames belonging to 2 classes.


In [23]:
test_generator.class_indices

{'ALL': 0, 'Normal': 1}

In [24]:
test_images, test_labels = next(test_generator)

In [25]:
m_test = test_images.shape[0]

print ("Number of testing samples: " + str(m_test))

print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))

Number of testing samples: 1867
test_images shape: (1867, 225, 225, 3)
test_labels shape: (1867,)


In [26]:
test_img = test_images.reshape(test_images.shape[0], -1)
print(test_img.shape)

(1867, 151875)


In [27]:
# test_y = np.reshape(test_labels[:,0], (1867,1))

In [28]:
test_labels[7]

0.0

In [29]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_generator.classes),
                                                  y=train_generator.classes
                                                 )

train_class_weights = dict(enumerate(class_weights))

In [30]:
baseline_model = models.Sequential()
baseline_model.add(layers.Dense(20, activation='relu', input_shape=(151875,))) # 2 hidden layers
baseline_model.add(layers.Dense(7, activation='relu'))
baseline_model.add(layers.Dense(5, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))

In [32]:
baseline_model.compile(optimizer='sgd',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

baseline_history = baseline_model.fit(train_img,
                                        train_labels,
                                        epochs=50,
                                        batch_size=32,
                                        validation_data=(val_img, val_labels),
                                        class_weight=train_class_weights,)

Epoch 1/50
263/263 [==============================] - 3s 10ms/step - loss: 0.5566 - accuracy: 0.7635 - val_loss: 0.6413 - val_accuracy: 0.6970
Epoch 2/50
263/263 [==============================] - 3s 10ms/step - loss: 0.5047 - accuracy: 0.7790 - val_loss: 0.6219 - val_accuracy: 0.6931
Epoch 3/50
263/263 [==============================] - 2s 9ms/step - loss: 0.4962 - accuracy: 0.7829 - val_loss: 0.5460 - val_accuracy: 0.7536
Epoch 4/50
263/263 [==============================] - 3s 10ms/step - loss: 0.4890 - accuracy: 0.7887 - val_loss: 0.4625 - val_accuracy: 0.8001
Epoch 5/50
263/263 [==============================] - 3s 10ms/step - loss: 0.4838 - accuracy: 0.7879 - val_loss: 0.5509 - val_accuracy: 0.7439
Epoch 6/50
263/263 [==============================] - 3s 10ms/step - loss: 0.4759 - accuracy: 0.7919 - val_loss: 0.4997 - val_accuracy: 0.7660
Epoch 7/50
263/263 [==============================] - 3s 10ms/step - loss: 0.4742 - accuracy: 0.7933 - val_loss: 0.4461 - val_accuracy: 0.8156


In [33]:
results_train = baseline_model.evaluate(train_img, train_labels)

263/263 [==============================] - 1s 4ms/step - loss: 0.4019 - accuracy: 0.8195


In [34]:
results_test = baseline_model.evaluate(test_img, test_labels)

59/59 [==============================] - 0s 5ms/step - loss: 1.5594 - accuracy: 0.3631


In [43]:
with open('data/test_data_copy/test_data_labels.csv', mode='r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    gt = {rows[0]:rows[1] for rows in reader}


# Importing the dataset

# setting path and list variables
path = 'data/test_data_copy/'
images = [] 
target = [] 

# loading dataset
for root, dirs, files in os.walk(path):
    for file in files:
        with open(os.path.join(root, file), "r") as auto:    
            im = cv2.imread(root+'\\'+file, 0)            
            images.append(im)
            target.append(gt[file])

[ WARN:0@1350.076] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('data/test_data_copy/\735.bmp'): can't open/read file: check file path/integrity


KeyError: '735.bmp'

In [70]:
# test_images_ALL = [x for x in test_df['new_names']test_df['labels'] == 'ALL']
# test_images_Normal = [x for x in test_df['new_names'] if test_df['labels'] == 'Normal']

test_split_df = test_df.groupby('labels')

In [71]:
test_df['labels'].value_counts()

Normal    1219
ALL        648
Name: labels, dtype: int64

In [74]:
test_images_ALL = test_split_df.get_group('ALL')
test_images_Normal = test_split_df.get_group('Normal')

In [75]:
test_images_ALL

,new_names,labels
3,4.bmp,ALL
6,7.bmp,ALL
7,8.bmp,ALL
12,13.bmp,ALL
13,14.bmp,ALL
...,...,...
1841,1842.bmp,ALL
1844,1845.bmp,ALL
1849,1850.bmp,ALL
1857,1858.bmp,ALL


In [76]:
test_images_Normal

,new_names,labels
0,1.bmp,Normal
1,2.bmp,Normal
2,3.bmp,Normal
4,5.bmp,Normal
5,6.bmp,Normal
...,...,...
1861,1862.bmp,Normal
1862,1863.bmp,Normal
1863,1864.bmp,Normal
1865,1866.bmp,Normal


In [87]:
test_image_file = 'data/test_data_copy/'
test_imgs_ALL = 'data/test_data_copy/ALL/'
test_imgs_Normal = 'data/test_data_copy/Normal/'

test_imgs_ALL = [x for x in test_images_ALL.new_names]
test_imgs_Normal = [x for x in test_images_Normal.new_names]

for file in test_image_file:
    if file in test_imgs_ALL:
        origin = os.path.join(test_image_file, file)
        destination = os.path.join(test_imgs_ALL, file)
        shutil.copyfile(origin, destination)
    elif file in test_imgs_Normal:
        origin = os.path.join(test_image_file, file)
        destination = os.path.join(test_imgs_Normal, file)
        shutil.copyfile(origin, destination)
        
# for img in imgs:
#     origin = os.path.join(data_training_ALL_dir, img)
#     destination = os.path.join(train_ALL, img)
#     shutil.copyfile(origin, destination)

In [85]:
test_imgs_ALL

['4.bmp',
 '7.bmp',
 '8.bmp',
 '13.bmp',
 '14.bmp',
 '21.bmp',
 '24.bmp',
 '26.bmp',
 '27.bmp',
 '28.bmp',
 '30.bmp',
 '31.bmp',
 '35.bmp',
 '38.bmp',
 '41.bmp',
 '43.bmp',
 '50.bmp',
 '54.bmp',
 '55.bmp',
 '57.bmp',
 '60.bmp',
 '61.bmp',
 '63.bmp',
 '66.bmp',
 '67.bmp',
 '69.bmp',
 '71.bmp',
 '72.bmp',
 '75.bmp',
 '77.bmp',
 '81.bmp',
 '82.bmp',
 '84.bmp',
 '87.bmp',
 '90.bmp',
 '92.bmp',
 '94.bmp',
 '104.bmp',
 '105.bmp',
 '106.bmp',
 '111.bmp',
 '115.bmp',
 '117.bmp',
 '118.bmp',
 '121.bmp',
 '123.bmp',
 '127.bmp',
 '138.bmp',
 '139.bmp',
 '140.bmp',
 '142.bmp',
 '144.bmp',
 '151.bmp',
 '154.bmp',
 '156.bmp',
 '158.bmp',
 '160.bmp',
 '164.bmp',
 '168.bmp',
 '171.bmp',
 '172.bmp',
 '173.bmp',
 '174.bmp',
 '181.bmp',
 '183.bmp',
 '187.bmp',
 '188.bmp',
 '192.bmp',
 '193.bmp',
 '197.bmp',
 '198.bmp',
 '201.bmp',
 '205.bmp',
 '206.bmp',
 '211.bmp',
 '212.bmp',
 '213.bmp',
 '214.bmp',
 '218.bmp',
 '220.bmp',
 '222.bmp',
 '225.bmp',
 '227.bmp',
 '229.bmp',
 '233.bmp',
 '235.bmp',
 '236.bm

In [88]:
for img in test_imgs_ALL:
    source = 'data/test_data_copy/' + img
    dest = 'data/test_data_copy/ALL/' + img
    os.rename(source, dest)

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.


In [89]:
for img in test_imgs_Normal:
    source = 'data/test_data_copy/' + img
    dest = 'data/test_data_copy/Normal/' + img
    os.rename(source, dest)

print("Source path renamed to destination path successfully.")

Source path renamed to destination path successfully.
